In [1]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
]


In [2]:
def get_completion(input):
    message = {"role": "user", "content": input}
    _history = history+[message]
    completion = client.chat.completions.create(
        model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
        messages=_history,
        temperature=0.7,
        stream=True,
    )
    return completion

In [3]:
def wrapper_chat_history(chat_history, history):
    chat_history = history[1:]
    return chat_history

In [4]:
def converse(message, chat_history):
    response = get_completion(input=message)
    user_msg = {"role": "user", "content": message}
    history.append(user_msg)
    ai_msg = {"role": "assistant", "content": ""}
    history.append(ai_msg)
    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            _tkn = chunk.choices[0].delta.content
            partial_message = partial_message + _tkn
            history[-1]["content"] = partial_message
            chat_history = wrapper_chat_history(chat_history, history)
            yield partial_message
    

In [5]:
import gradio as gr

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
]

demo = gr.ChatInterface(fn=converse)

demo.launch(share=False)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
